Si se inspecciona el sitio Booking.com, uno se puede dar cuenta que las clases de los elementos de la página estan 
definidas por variables aleatorias de 10 caracteres, lo que genera bastantes problemas si uno quiere buscar un elemento 
determinado para realizar scrape. Además, se tienen instancias donde multiples clases definen un mismo objeto, lo
que provoca un mayor tiempo de carga de la página y mayor dificultad en la lectura del HTML. 

Ahora bien, para obtener los objetos a los cuales se les quiere realizar scrape, se debe revisar el HTML mediante Inspeccionar elemento en un navegador (por ejemplo Chrome y Edge, basados en Chromium) e identificar las clases en 
HTML que definan un valor de texto determinado, como por ejemplo en el caso de Booking.com

In [ ]:
<div data-testid="title" class="fcab3ed991 a23c043802">Ruka Peumayen Caburgua</div>

el cual define el nombre del hotel Ruka Peumayen Caburgua en la clase "fcab3ed991 a23c043802" y posee alias "title".
Una forma en que se pueden realizar extracciones de esta pagina web es utilizando la libreria Scrapy para Python,
la cual se puede instalar de diferentes formas siguiendo el tutorial disponible en https://docs.scrapy.org/en/latest/intro/install.html. 
Sin embargo, tambien hay diferentes formas en las que se puede programar una extraccion usando Scrapy, siendo la forma más popular mediante la creacion de un proyecto Scrapy en una terminal (Powershell de Windows, Linux, o de Anaconda) por medio del comando:
1. En terminal de Linux:              scrapy startproject projectname 
2. En terminal de Anaconda:  python -m scrapy startproject projectname 

El comando creará una carpeta projectname con archivos de Python predeterminados que serán utilizados por Scrapy, los cuales se deben editar en función de lo que uno requiere hacer. La otra forma de realizar la extraccion es directamente desde Jupyter Notebook sin tener que crear un proyecto de Scrapy. Esta ultima forma es la que será utilizada para definir la extracción de Booking.com en el siguiente notebook.

Generalmente, en Scrapy uno debe definir una clase que crear un objeto Item, el cual permite guardar los datos no estructurados de una página web en un objeto estructurado con Fields definidos por Scrapy, los cuales pueden ser listas o diccionarios. Un ejemplo de Item y Fields en Scrapy es de la siguiente forma:

In [ ]:
import scrapy
from scrapy.item import Item, Field
class BookingItem(Item):
    nombre = Field()
    nota = Field()
    lugar = Field()

El cual define la clase BookingItem donde se guardaran los parametros nombre, nota y lugar, que significan el nombre de 
un Hotel, la calificación que tiene, y el lugar donde se encuentra, datos que serán extraidos de Booking.com directamente.
Dado esto, se debe definir la clase que utiliza la "araña" de Scrapy, el cual es el objeto cuyo propósito es extraer los
datos de una página web. 
Para el caso de Booking.com, se debe tomar en cuenta que la interfaz solo puede mostrar un máximo
40 páginas con 25 hoteles cada una, por lo que una busqueda retorna máximo 1000 hoteles en total, agregando que en la URL de
Booking.com luego de buscar un determinado hotel, además de ser innecesariamente larga, se pueden editar fácilmente los
parámetros de busqueda lo que permite flexibilidad al querer hacer scrape a un conjunto de hoteles determinado.
Otro problema de Booking.com es que solo se pueden ver los precios si se fija una fecha de entrada y una fecha de salida

La clase Spider de Scrapy se puede definir de la siguiente forma:

In [ ]:
class BookingSpider(scrapy.Spider):
    name = "booking"
    allowed_domains = ["booking.com"]
    #Direccion(es) donde realizar web scraping
    start_urls = ['https://www.booking.com/hotel/cl']
    #Configuracion de pipelines
    custom_settings = {
    }
    def parse(self, response):
        for question in response.css('%'):
            item = BookingItem()
            item['nombre'] = question.css('%').extract_first()
            item['nota'] = question.css('%').extract_first()
            item['lugar'] = question.css('%').extract_first()
            yield item

Donde *custom_settings* es equivalente al archivo settings.py al realizarlo mediante la creación de un proyecto, en el
se pueden editar los pipelines, que son parametros que definen, por ejemplo, como y donde guardar la base de datos además
de datos adicionales para que el scrape sea correcto.
La ultima parte de la función tambien se puede hacer mediante un diccionario en vez de un Item de Scrapy, quedando
de la siguiente forma:

In [ ]:
def parse(self, response):
        #Iterar en todos los objetos con la clase que define un hotel
        for question in response.css('%'):
            yield{ 
            'nombre' : question.css('%').extract_first(),
            'nota' : question.css('%').extract_first(),
            'lugar' : question.css('%').extract_first(),
            }

Es importante notar que *response.css()* (tambien puede ser *response.xpath()*) es denominado Selector en Scrapy, y es 
necesario para adquirir los datos en HTML y convertirlos a un formato reconocible por Python. Por ejemplo, para el HTML
que define el nombre en el caso de Booking.com, su conversion a CSS queda de la siguiente forma:

In [ ]:
<div data-testid="title" class="fcab3ed991 a23c043802">Ruka Peumayen Caburgua</div>
css -> 'div.fcab3ed991.a23c043802'

Lo que implica que para buscar el nombre del hotel en el fragmento de codigo expuesto anteriormente, se debe escribir:
>'nombre' : question.css('div.fcab3ed991.a23c043802::text').extract_first()

el cual extrae el texto "Ruka Peumayen Caburgua" asignado a la clase "fcab3ed991 a23c043802".